## data crawling

### primary link and ID

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

headers = {
    'Accept': 'text/html',
    'Accept-Language': 'en',
}

n = 200
path = 'data/steam_link.csv'

def getgamelist(n):
    linklist=[]
    IDlist = []
    for pagenum in range(1, n):
        r = requests.get('https://store.steampowered.com/search/?ignore_preferences=1&category1=998&filter=globaltopsellers&page=%d'%pagenum,headers=headers)
        soup = BeautifulSoup(r.text, 'lxml')
        soups= soup.find_all(href=re.compile(r"https://store.steampowered.com/app/"),class_="search_result_row ds_collapse_flag")
        for i in soups:
            i = i.attrs
            i = i['href']
            link = re.search('https://store.steampowered.com/app/(\d*?)/',i).group()
            ID = re.search('https://store.steampowered.com/app/(\d*?)/(.*?)/', i).group(1)
            linklist.append(link)
            IDlist.append(ID)
        print('completed '+str(pagenum)+' page, currently total '+str(len(linklist)))
    return linklist,IDlist

def getdf(n):
    linklist,IDlist = getgamelist(n)
    df = pd.DataFrame(list(zip(linklist,IDlist)),
               columns =['Link', 'ID'])
    return df

if __name__ == "__main__":
    df = getdf(n)
    df.to_csv(path,index=False)

completed 1 page, currently total 25


### get the details in link

In [2]:
def gamename(soup): #name
    try:
        a = soup.find(class_="apphub_AppName")
        k = str(a.string)
    except:
        a = soup.find(class_="apphub_AppName")
        k = str(a.text)
    return k

def gameprice(soup):#price
    try:
        a = soup.findAll(class_="discount_original_price")
        for i in a:
            if re.search('$|free', str(i),re.IGNORECASE):
                a = i
        k = str(a.string).replace('	', '').replace('\n', '').replace('\r', '').replace(' ', '')
    except:
        a = soup.findAll(class_="game_purchase_price price")
        for i in a:
            if re.search('$|free', str(i),re.IGNORECASE):
                a = i
        k = str(a.string).replace('	', '').replace('\n', '').replace('\r', '').replace(' ', '')
    return k

def taglist(soup):#tag list
    list1=[]
    a = soup.find_all(class_="app_tag")
    for i in a:
        k = str(i.string).replace('	', '').replace('\n', '').replace('\r', '')
        if k == '+':
            pass
        else:
            list1.append(k)
    list1 = str(';'.join(list1))
    return list1

def getdate(soup):   #release date
    a = soup.find(class_="date")
    k = str(a.string)
    return k

def reviewsummary(soup):   #reviews
    a = soup.find(class_="summary column")
    try:
        k = str(a.span.string)
    except:
        k=str(a.text)
    return k

def userreviewsrate(soup):  #rate
    a = soup.find(class_="user_reviews_summary_row")
    k = str((a.attrs)['data-tooltip-html'])
    return k

def developer(soup):   #developer
    a = soup.find(id="developers_list")
    k = str(a.a.string)
    return k

def findplatform(soup): #platform
    sysreq_tabs_div = soup.find('div', {'class': 'sysreq_tabs'})
    if sysreq_tabs_div:
        platforms_divs = sysreq_tabs_div.find_all('div', {'class': 'sysreq_tab'})
        return [div.get('data-os') for div in platforms_divs if div.get('data-os')]

    sysreq_contents_div = soup.find('div', {'class': 'sysreq_contents'})
    if sysreq_contents_div:
        platforms_divs = sysreq_contents_div.find_all('div', {'class': 'game_area_sys_req sysreq_content active'})
        return [div.get('data-os') for div in platforms_divs if div.get('data-os')]

    return ['No platforms found']



def getgenre(soup): #genre
    genres_div = soup.find('div', {'id': 'genresAndManufacturer'})
    if genres_div:
        genre_b_tag = genres_div.find('b', text='Genre:')
        if genre_b_tag:
            genre_span = genre_b_tag.find_next_sibling('span')
            if genre_span:
                genres = genre_span.get_text(strip=True)
            else:
                genres = 'Genre not found'
        else:
            genres = 'Genre tag not found'
    else:
        genres = 'Genres and Manufacturer div not found'
    return genres


def getdetail(x):
    tag, date, reviews, rate, dev, name, price, genre, platform = ' ', ' ', ' ', ' ', ' ', ' ', ' ', None, None
    global count
    try:
        r = requests.get(x['Link'], headers=headers,timeout=10)
    except:
        print('server no reply 1')
        try:
            r = requests.get(x['Link'], headers=headers,timeout=10)
        except:
            print('server no reply 2')
            try:
                r = requests.get(x['Link'], headers=headers,timeout=10)
            except:
                print('server no reply 3')

    try:
        soup = BeautifulSoup(r.text, 'lxml')
        name = gamename(soup)
        tag = taglist(soup)
        date = getdate(soup)
        reviews = reviewsummary(soup)
        rate = userreviewsrate(soup)
        dev = developer(soup)
        price = gameprice(soup)
        genre = getgenre(soup)
        platform = findplatform(soup)
        print('complete: '+ name + str(x['ID']) +' num: %d'%count)
    except:
        print('not complete: '+ str(x['ID']) +' num: %d'%count)
        price = 'error'

    count += 1
    return name, price, tag, date, reviews, rate, dev, genre, platform

if __name__ == "__main__":
    df1 = pd.read_csv(path)
    count = 1
    df1['detail'] = df1.apply(getdetail, axis=1)
    df1['name'] = df1.apply(lambda x: x['detail'][0], axis=1)
    df1['price'] = df1.apply(lambda x: x['detail'][1], axis=1)
    df1['label'] = df1.apply(lambda x: x['detail'][2], axis=1)
    df1['release_date'] = df1.apply(lambda x: x['detail'][3], axis=1)
    df1['recent_evalution'] = df1.apply(lambda x: x['detail'][4], axis=1)
    df1['Recent_quantity_praise_rate'] = df1.apply(lambda x: x['detail'][5], axis=1)
    df1['developers'] = df1.apply(lambda x: x['detail'][6], axis=1)
    df1['genre'] = df1.apply(lambda x: x['detail'][7], axis=1)
    df1['platform'] = df1.apply(lambda x: x['detail'][8], axis=1)
    df1 = df1.drop(columns=['detail'])
    
    detail_path = 'data/steam_detail.csv'
    df1.to_csv(detail_path,index=False,encoding='utf-8-sig')
    print('Completed all')

C:\Users\12073\AppData\Local\Temp\ipykernel_41240\3349578442.py:78: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  genre_b_tag = genres_div.find('b', text='Genre:')


complete: Lethal Company1966720 num: 1
complete: Counter-Strike 2730 num: 2
complete: PUBG: BATTLEGROUNDS578080 num: 3
complete: Call of Duty®1938090 num: 4
complete: Baldur's Gate 31086940 num: 5
complete: American Truck Simulator270880 num: 6
complete: EA SPORTS FC™ 242195250 num: 7
complete: Destiny 21085660 num: 8
complete: Apex Legends™1172470 num: 9
complete: Call to Arms - Gates of Hell: Ostfront400750 num: 10
complete: TEVI2230650 num: 11
server no reply 1
complete: Dead by Daylight381210 num: 12
complete: Lost Ark1599340 num: 13
complete: Last Train Home1469610 num: 14
complete: NARAKA: BLADEPOINT1203220 num: 15
complete: Horizon Zero Dawn™ Complete Edition1151640 num: 16
complete: The Elder Scrolls® Online306130 num: 17
complete: War Thunder236390 num: 18
complete: ARK: Survival Ascended2399830 num: 19
complete: Yu-Gi-Oh! Master Duel1449850 num: 20
complete: Dragon's Dogma 22054970 num: 21
complete: Warframe230410 num: 22
complete: Cyberpunk 20771091500 num: 23
complete: Euro

## data cleaning

In [3]:
import pandas as pd
import re

In [4]:
df = pd.read_csv(detail_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Link                         25 non-null     object
 1   ID                           25 non-null     int64 
 2   name                         25 non-null     object
 3   price                        25 non-null     object
 4   label                        25 non-null     object
 5   release_date                 25 non-null     object
 6   recent_evalution             25 non-null     object
 7   Recent_quantity_praise_rate  25 non-null     object
 8   developers                   25 non-null     object
 9   genre                        25 non-null     object
 10  platform                     25 non-null     object
dtypes: int64(1), object(10)
memory usage: 2.3+ KB


In [5]:
def price(x):
    try:
        pricenum = float(str(x['price']).replace('$',''))
    except:
        pricenum = 0
    return pricenum
df = df[df['price'] != 'error'].reset_index(drop=True)
df['price'] = df.apply(lambda x:price(x),axis=1)
df['price'] = pd.to_numeric(df['price'])
df['price'].head(10)

0     9.99
1    14.99
2     0.00
3    30.00
4     9.99
5     1.99
6    99.99
7    24.99
8     0.00
9    52.97
Name: price, dtype: float64

In [6]:
def getreviewsnum(x):
    x1 = x['Recent_quantity_praise_rate'].replace(',','')
    x2 = x['recent_evalution']
    if re.search('(\d*%) of the (\d*) user reviews for this game are positive in the past 30 days.',x1):
        num = re.search('(\d*%) of the (\d*) user reviews for this game are positive in the past 30 days.',x1).group(2)
    elif re.search('(\d*%) of the (\d*) user reviews for this game are positive.',x1):
        num = re.search('(\d*%) of the (\d*) user reviews for this game are positive.',x1).group(2)
    elif re.search('(\d*) user reviews',x2):
        num = re.search('(\d*) user reviews',x2).group(1)
    elif re.search('(\d*%) of the (\d*) user reviews in the last 30 days are positive.*',x1):
        num = re.search('(\d*%) of the (\d*) user reviews in the last 30 days are positive',x1).group(2)
    elif re.search('(\d*%) of the (\d*) user reviews for this game are positive.*',x1):
        num = re.search('(\d*%) of the (\d*) user reviews for this game are positive',x1).group(2)
    else:
        print(x1)
        num = '0'
    
    return num
def getreviewsrate(x):
    x = x['Recent_quantity_praise_rate'].replace(',','')
    if re.search('(\d*%) of the (\d*) user reviews for this game are positive in the past 30 days.',x):
        rate = re.search('(\d*%) of the (\d*) user reviews for this game are positive in the past 30 days.',x).group(1)
    elif re.search('(\d*%) of the (\d*) user reviews for this game are positive.',x):
        rate = re.search('(\d*%) of the (\d*) user reviews for this game are positive.',x).group(1)
    elif re.search('(\d*%) of the (\d*) user reviews in the last 30 days are positive.*',x):
        rate = re.search('(\d*%) of the (\d*) user reviews in the last 30 days are positive',x).group(1)
    elif re.search('(\d*%) of the (\d*) user reviews for this game are positive.*',x):
        rate = re.search('(\d*%) of the (\d*) user reviews for this game are positive',x).group(1)
    else:
        print(x)
        rate = '0%'
    return rate.replace('%','')
df['num_reviews']=df.apply(lambda x:getreviewsnum(x),axis=1)
df['positive_review_rate']=df.apply(lambda x:getreviewsrate(x),axis=1)
df.head(10)

No user reviews


,Link,ID,name,price,label,release_date,recent_evalution,Recent_quantity_praise_rate,developers,genre,platform,num_reviews,positive_review_rate
0,https://store.steampowered.com/app/1966720/,1966720,Lethal Company,9.99,Early Access;Online Co-Op;Horror;First-Person;...,"Oct 23, 2023",Overwhelmingly Positive,"98% of the 93,290 user reviews in the last 30 ...",Zeekerss,"Action,Adventure,Indie,Early Access",['win'],93290,98
1,https://store.steampowered.com/app/730/,730,Counter-Strike 2,14.99,FPS;Shooter;Multiplayer;Competitive;Action;Tea...,"Aug 21, 2012",Mostly Positive,"73% of the 65,385 user reviews in the last 30 ...",Valve,"Action,Free to Play","['win', 'linux']",65385,73
2,https://store.steampowered.com/app/578080/,578080,PUBG: BATTLEGROUNDS,0.00,Survival;Shooter;Battle Royale;Multiplayer;FPS...,"Dec 21, 2017",Mostly Positive,"76% of the 21,868 user reviews in the last 30 ...","KRAFTON, Inc.","Action,Adventure,Massively Multiplayer,Free to...",['win'],21868,76
3,https://store.steampowered.com/app/1938090/,1938090,Call of Duty®,30.00,FPS;Multiplayer;Shooter;Action;Singleplayer;Mi...,"Oct 27, 2022",Mixed,"41% of the 21,384 user reviews in the last 30 ...",Infinity Ward,Action,['win'],21384,41
4,https://store.steampowered.com/app/1086940/,1086940,Baldur's Gate 3,9.99,RPG;Choices Matter;Character Customization;Sto...,"Aug 3, 2023",Overwhelmingly Positive,"98% of the 88,667 user reviews in the last 30 ...",Larian Studios,"Adventure,RPG,Strategy","['win', 'mac']",88667,98
5,https://store.steampowered.com/app/270880/,270880,American Truck Simulator,1.99,Automobile Sim;Driving;Open World;Transportati...,"Feb 2, 2016",Overwhelmingly Positive,"98% of the 2,489 user reviews in the last 30 d...",SCS Software,"Indie,Simulation","['win', 'mac', 'linux']",2489,98
6,https://store.steampowered.com/app/2195250/,2195250,EA SPORTS FC™ 24,99.99,Sports;Football (Soccer);Controller;PvP;Compet...,"Sep 28, 2023",Mixed,"60% of the 9,049 user reviews in the last 30 d...",EA Canada & EA Romania,"Simulation,Sports",['win'],9049,60
7,https://store.steampowered.com/app/1085660/,1085660,Destiny 2,24.99,Free to Play;Open World;FPS;Looter Shooter;MMO...,"Oct 1, 2019",Mixed,"57% of the 6,128 user reviews in the last 30 d...",Bungie,"Action,Adventure,Free to Play",['win'],6128,57
8,https://store.steampowered.com/app/1172470/,1172470,Apex Legends™,0.00,Free to Play;Battle Royale;Multiplayer;Shooter...,"Nov 4, 2020",Mostly Positive,"75% of the 28,696 user reviews in the last 30 ...",Respawn Entertainment,"Action,Adventure,Free to Play",['win'],28696,75
9,https://store.steampowered.com/app/400750/,400750,Call to Arms - Gates of Hell: Ostfront,52.97,World War II;Strategy;Real Time Tactics;RTS;Wa...,"Jun 11, 2021",Very Positive,"84% of the 1,000 user reviews in the last 30 d...",Barbedwire Studios,"Action,Simulation,Strategy",['win'],1000,84


In [7]:
len(df)

25

In [8]:
df = df[df['Recent_quantity_praise_rate'] != 'Need more user reviews to generate a score'].reset_index(drop=True)
df = df[df['Recent_quantity_praise_rate'] != 'No user reviews'].reset_index(drop=True)
len(df)

24

In [9]:
df['num_reviews'] = pd.to_numeric(df['num_reviews'])
df['positive_review_rate'] = pd.to_numeric(df['positive_review_rate'])

In [10]:
convert_dict = {'release_date': 'datetime64'}
df = df.astype(convert_dict)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Link                         24 non-null     object        
 1   ID                           24 non-null     int64         
 2   name                         24 non-null     object        
 3   price                        24 non-null     float64       
 4   label                        24 non-null     object        
 5   release_date                 24 non-null     datetime64[ns]
 6   recent_evalution             24 non-null     object        
 7   Recent_quantity_praise_rate  24 non-null     object        
 8   developers                   24 non-null     object        
 9   genre                        24 non-null     object        
 10  platform                     24 non-null     object        
 11  num_reviews                  24 non-null     in

In [11]:
selected_columns = ['ID', 'name', 'price', 'label', 'release_date', 'developers', 'genre', 'platform', 'num_reviews', 'positive_review_rate']
df = df[selected_columns]
df.head(10)

,ID,name,price,label,release_date,developers,genre,platform,num_reviews,positive_review_rate
0,1966720,Lethal Company,9.99,Early Access;Online Co-Op;Horror;First-Person;...,2023-10-23,Zeekerss,"Action,Adventure,Indie,Early Access",['win'],93290,98
1,730,Counter-Strike 2,14.99,FPS;Shooter;Multiplayer;Competitive;Action;Tea...,2012-08-21,Valve,"Action,Free to Play","['win', 'linux']",65385,73
2,578080,PUBG: BATTLEGROUNDS,0.00,Survival;Shooter;Battle Royale;Multiplayer;FPS...,2017-12-21,"KRAFTON, Inc.","Action,Adventure,Massively Multiplayer,Free to...",['win'],21868,76
3,1938090,Call of Duty®,30.00,FPS;Multiplayer;Shooter;Action;Singleplayer;Mi...,2022-10-27,Infinity Ward,Action,['win'],21384,41
4,1086940,Baldur's Gate 3,9.99,RPG;Choices Matter;Character Customization;Sto...,2023-08-03,Larian Studios,"Adventure,RPG,Strategy","['win', 'mac']",88667,98
5,270880,American Truck Simulator,1.99,Automobile Sim;Driving;Open World;Transportati...,2016-02-02,SCS Software,"Indie,Simulation","['win', 'mac', 'linux']",2489,98
6,2195250,EA SPORTS FC™ 24,99.99,Sports;Football (Soccer);Controller;PvP;Compet...,2023-09-28,EA Canada & EA Romania,"Simulation,Sports",['win'],9049,60
7,1085660,Destiny 2,24.99,Free to Play;Open World;FPS;Looter Shooter;MMO...,2019-10-01,Bungie,"Action,Adventure,Free to Play",['win'],6128,57
8,1172470,Apex Legends™,0.00,Free to Play;Battle Royale;Multiplayer;Shooter...,2020-11-04,Respawn Entertainment,"Action,Adventure,Free to Play",['win'],28696,75
9,400750,Call to Arms - Gates of Hell: Ostfront,52.97,World War II;Strategy;Real Time Tactics;RTS;Wa...,2021-06-11,Barbedwire Studios,"Action,Simulation,Strategy",['win'],1000,84


In [73]:
df_cleaned = df.dropna()
df_cleaned.to_csv('data/steam_cleaned.csv', index=False, encoding='utf-8-sig')